# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    He = ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))
    atoms = [He => [[1/2; 1/2; 1/2]]]  # Helium at the center of the box

    model = model_DFT(lattice, atoms, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770841400796         NaN   2.99e-01   0.80    9.0
  2   -2.772147214716   -1.31e-03   4.75e-02   0.80    1.0
  3   -2.772169844520   -2.26e-05   4.37e-03   0.80    2.0
  4   -2.772170722481   -8.78e-07   9.41e-05   0.80    2.0
  5   -2.772170723001   -5.20e-10   1.11e-05   0.80    2.0
n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770449063060         NaN   2.96e-01   0.80    8.0
  2   -2.772046007640   -1.60e-03   5.12e-02   0.80    1.0
  3   -2.772083341695   -3.73e-05   1.95e-03   0.80    2.0
  4   -2.772083416731   -7.50e-08   6.59e-05   0.80    2.0
  5   -2.772083417773   -1.04e-09   2.18e-05   0.80    3.0


1.7737964981538545

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770741020294         NaN   2.98e-01   0.80    8.0
  2   -2.772052207936   -1.31e-03   4.85e-02   0.80    1.0
  3   -2.772082687447   -3.05e-05   2.91e-03   0.80    1.0
  4   -2.772083395277   -7.08e-07   3.63e-04   0.80    2.0
  5   -2.772083415946   -2.07e-08   1.32e-04   0.80    2.0
  6   -2.772083417803   -1.86e-09   5.62e-06   0.80    2.0

Polarizability via ForwardDiff:       1.7725473344327516
Polarizability via finite difference: 1.7737964981538545
